# Tool Calls mit LangChain und LLMs

In diesem Notebook lernen wir, wie wir eigene Tools erstellen und in LLMs integrieren können. Tool Calls ermöglichen es, die Fähigkeiten von LLMs durch externe Funktionen und Dienste zu erweitern.

## 1. Grundlegende Importe und Setup

In [1]:
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)
from helpers import llm
from langchain.tools import tool
from langchain.schema import StrOutputParser
from langchain_core.messages import BaseMessage
import json

# OpenAI API Wrapper erstellen
model = llm(model="gpt-4o-mini")

## 2. Ein einfaches Tool erstellen

In [12]:
@tool
def personen_info(name: str):
    """Gibt Informationen über eine Person zurück.
    
    Args:
        name: Der Name der Person
        
    Returns:
        Informationen über die Person
    """
    personen = {
        "fritz karuugaa": "Fritz Karuugaa ist ein Software-Entwickler.",
        "albert einstein": "Albert Einstein (1879-1955) war ein theoretischer Physiker und Entwickler der Relativitätstheorie.",
        "marie curie": "Marie Curie (1867-1934) war eine Physikerin und Chemikerin, die zweimal den Nobelpreis erhielt."
    }
    
    return personen.get(name.lower(), f"Keine Informationen über {name} verfügbar.")

# Testen des Tools
print(personen_info.invoke("Fritz Karuugaa"))
print(personen_info.invoke("Albert Einstein"))
print(personen_info.invoke("Marie Curie"))

Fritz Karuugaa ist ein Software-Entwickler.
Albert Einstein (1879-1955) war ein theoretischer Physiker und Entwickler der Relativitätstheorie.


## 3. Tools an ein LLM binden

In [13]:
# Tools als Liste bereitstellen (auch wenn es nur eines ist)
tools = [personen_info]

# Tools an das LLM binden
# Wichtig: Hierbei werden die Tool-Beschreibungen automatisch in den Kontext des LLMs eingefügt
llm_with_tools = model.bind_tools(tools)

# Prompt Template erstellen
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(
            "Du bist ein hilfsbereicher Assistent, der Fragen zu Personen beantwortet."
        ),
        HumanMessagePromptTemplate.from_template("{input}"),
        MessagesPlaceholder("intermediate_steps"),
    ]
)

# Einfache LangChain Expression Language (LCEL) Chain erstellen
chain = prompt | llm_with_tools

## 4. Das Tool mit einer einfachen Anfrage testen

**Kleiner Helfer um Chain antworten schön anziegen zu lassen**

In [ ]:
# Response printer function
def print_response(response):
    if isinstance(response, BaseMessage):
        response.pretty_print()
    elif isinstance(response, (str, list, dict)):
        print(json.dumps(response, indent=2))

In [21]:
# Einfache Anfrage an das LLM mit Tool-Unterstützung
response = chain.invoke({"input": "Wer ist Marie Curie?"})
print(response)
print(response.tool_calls[0])
print_response(response)

content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_4UUcUv20yIegrQcV47J2knU3', 'function': {'arguments': '{"name":"Marie Curie"}', 'name': 'personen_info'}, 'type': 'function'}]} response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_726d488742'} id='run-b9c99c1b-290c-4c6c-8923-b376fffedb06-0' tool_calls=[{'name': 'personen_info', 'args': {'name': 'Marie Curie'}, 'id': 'call_4UUcUv20yIegrQcV47J2knU3', 'type': 'tool_call'}]


## 5. Währungsumrechnungs-Tool erstellen

Jetzt implementieren wir ein Tool zur Währungsumrechnung.

In [22]:
@tool
def waehrungsumrechnung(betrag: float, von_währung: str, zu_währung: str):
    """Rechnet einen Geldbetrag von einer Währung in eine andere um.
    
    Args:
        betrag: Der umzurechnende Geldbetrag
        von_währung: Quellwährung (z.B. "EUR", "USD", "GBP", "JPY")
        zu_währung: Zielwährung (z.B. "EUR", "USD", "GBP", "JPY")
        
    Returns:
        Eine Zeichenkette mit dem umgerechneten Betrag
    """
    # Einfache Wechselkurse (in der Realität würde man eine API verwenden)
    kurse = {"EUR": 1.0, "USD": 1.08, "GBP": 0.85, "JPY": 163.2, "CHF": 0.97}
    
    # Prüfen, ob die Währungen unterstützt werden
    if von_währung not in kurse or zu_währung not in kurse:
        return f"Fehler: Eine der angegebenen Währungen wird nicht unterstützt. Verfügbare Währungen: {', '.join(kurse.keys())}"
    
    # Umrechnung durchführen
    ergebnis = betrag * (kurse[zu_währung] / kurse[von_währung])
    
    # Ergebnis zurückgeben
    return f"{betrag} {von_währung} = {ergebnis:.2f} {zu_währung}"

# Testen des Tools
print(währungsumrechnung(100, "EUR", "USD"))
print(währungsumrechnung(50, "GBP", "EUR"))

TypeError: BaseTool.__call__() takes from 2 to 3 positional arguments but 4 were given

## 6. Mehrere Tools kombinieren

In [24]:
# Wetterinformations-Tool erstellen
@tool
def wetter_info(ort: str):
    """Gibt Wetterinformationen für einen bestimmten Ort zurück.
    
    Args:
        ort: Der Name der Stadt oder des Ortes
        
    Returns:
        Eine Beschreibung des aktuellen Wetters
    """
    # Beispieldaten (in der Realität würde man eine Wetter-API verwenden)
    wetterdaten = {
        "berlin": "Berlin: 22°C, sonnig",
        "hamburg": "Hamburg: 18°C, leicht bewölkt, leichter Wind",
        "münchen": "München: 20°C, vereinzelte Wolken",
        "köln": "Köln: 21°C, wolkenlos",
        "frankfurt": "Frankfurt: 23°C, heiter bis wolkig"
    }
    
    return wetterdaten.get(ort.lower(), f"Keine Wetterdaten für {ort} verfügbar.")

# Alle Tools kombinieren
multi_tools = [personen_info, waehrungsumrechnung, wetter_info]

# LLM mit allen Tools ausstatten
llm_with_multi_tools = model.bind_tools(multi_tools)

# Neues Prompt-Template erstellen
multi_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(
            """Du bist ein vielseitiger Assistent, der mit verschiedenen Werkzeugen ausgestattet ist, um Fragen zu beantworten.
            Verwende das passende Werkzeug, um die Anfrage des Nutzers zu beantworten."""
        ),
        HumanMessagePromptTemplate.from_template("{input}"),
        MessagesPlaceholder("intermediate_steps"),
    ]
)

# Neue Chain erstellen
multi_chain = multi_prompt | llm_with_multi_tools

## 7. Test mit verschiedenen Anfragen

In [25]:
# Test 1: Personeninfo
response1 = multi_chain.invoke({"input": "Kannst du mir etwas über Albert Einstein sagen?"})
print("Antwort 1:\n")
print(response1)
print_response(response1)
print("\n" + "-"*50 + "\n")

# Test 2: Währungsumrechnung
response2 = multi_chain.invoke({"input": "Wie viel sind 200 Euro in US-Dollar?","intermediate_steps":[]})
print("Antwort 2:\n")
print(response2)
print_response(response2)
print("\n" + "-"*50 + "\n")

# Test 3: Wetterinfo
response3 = multi_chain.invoke({"input": "Wie ist das Wetter in Berlin?","intermediate_steps":[]})
print("Antwort 3:\n")
print(response3)
print_response(response3)

BadRequestError: Error code: 400 - {'error': {'message': "Invalid 'tools[1].function.name': string does not match pattern. Expected a string that matches the pattern '^[a-zA-Z0-9_-]+$'.", 'type': 'invalid_request_error', 'param': 'tools[1].function.name', 'code': 'invalid_value'}}

## 8. Integration mit LangGraph

Für komplexere Anwendungen können wir Tool Calls in einen LangGraph-Workflow integrieren. Hier zeigen wir einen einfachen Ansatz.

In [26]:
import operator
from typing import Annotated, TypedDict
from langchain_core.agents import AgentActionMessageLog, AgentFinish
from langchain_core.messages import ToolMessage
from langchain_core.runnables import RunnableLambda
from langgraph.graph import END, StateGraph

# State für den Graphen definieren
class AgentState(TypedDict):
    input: str
    intermediate_steps: Annotated[list[tuple[AgentActionMessageLog, str]], operator.add]
    answer: str

In [27]:
# Entscheidungsfunktion: Soll ein Tool ausgeführt werden oder sind wir fertig?
def should_continue(state):
    if not state["intermediate_steps"]:
        return "continue"
    
    last_step = state["intermediate_steps"][-1]
    if isinstance(last_step, AgentActionMessageLog) and last_step.tool_calls:
        return "continue"
    return "end"

# Funktion zum Aufrufen des LLM/Agenten
def call_model(state, config):
    response = multi_chain.invoke(state, config=config)
    
    if isinstance(response, AgentFinish):
        return {"answer": response.answer}
    else:
        return {"intermediate_steps": [response]}

# Funktion zur Ausführung eines einzelnen Tools
def _invoke_tool(tool_call):
    tool_map = {tool.name: tool for tool in multi_tools}
    tool = tool_map[tool_call["name"]]
    return ToolMessage(tool.invoke(tool_call["args"]), tool_call_id=tool_call["id"])

# Wrapper für die Tool-Ausführung
tool_executor = RunnableLambda(_invoke_tool)

# Funktion zum Aufrufen aller benötigten Tools
def call_tools(state):
    last_message = state["intermediate_steps"][-1]
    return {"intermediate_steps": tool_executor.batch(last_message.tool_calls)}

In [28]:
# Graph-Workflow erstellen
workflow = StateGraph(AgentState)
workflow.add_node("agent", call_model)
workflow.add_node("action", call_tools)
workflow.set_entry_point("agent")

# Bedingungen für Verzweigungen hinzufügen
workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "action",  # Tool ausführen
        "end": END,             # Fertig
    },
)
workflow.add_edge("action", "agent")

# Graph kompilieren
graph = workflow.compile()

## 9. Graph-Workflow testen

In [29]:
# Komplexere Anfrage, die mehrere Tools benötigen könnte
result = graph.invoke({
    "input": "Ich möchte heute in Berlin spazieren gehen. Wie ist das Wetter dort? Außerdem will ich etwas Geld wechseln - wie viel bekomme ich für 150 EUR in Pfund?",
    "intermediate_steps": []
})

print("Finales Ergebnis:")
if "answer" in result:
    print(result["answer"])
else:
    print("Keine endgültige Antwort gefunden.")

BadRequestError: Error code: 400 - {'error': {'message': "Invalid 'tools[1].function.name': string does not match pattern. Expected a string that matches the pattern '^[a-zA-Z0-9_-]+$'.", 'type': 'invalid_request_error', 'param': 'tools[1].function.name', 'code': 'invalid_value'}}

## 10. Übungsaufgaben

1. Erstellen Sie ein neues Tool, das mathematische Berechnungen durchführen kann (z.B. Addition, Subtraktion, Multiplikation, Division).
2. Fügen Sie das Tool zu den bestehenden Tools hinzu und testen Sie es mit verschiedenen Anfragen.
3. Erweitern Sie das Währungsumrechnungs-Tool um mindestens zwei weitere Währungen.
4. Bonus: Implementieren Sie ein Tool, das einen kurzen Text zusammenfassen kann.

In [ ]:
# Ihre Lösung für das Mathe-Tool
@tool
def mathe_berechnung(operation: str, zahl1: float, zahl2: float):
    """Führt eine mathematische Berechnung mit zwei Zahlen durch.
    
    Args:
        operation: Die durchzuführende Operation ("addition", "subtraktion", "multiplikation", "division")
        zahl1: Die erste Zahl
        zahl2: Die zweite Zahl
        
    Returns:
        Das Ergebnis der Berechnung
    """
    # Ihre Implementierung hier
    pass

## 11. Zusammenfassung

In diesem Notebook haben wir gelernt:

- Wie man eigene Tools mit dem `@tool`-Decorator erstellt
- Wie man diese Tools an ein LLM bindet
- Wie man mehrere Tools kombiniert für vielseitige Anfragen
- Wie man Tools in einen LangGraph-Workflow integriert

Tool Calls sind ein mächtiges Konzept, um die Fähigkeiten von LLMs zu erweitern und sie mit externen Funktionen, Daten und APIs zu verbinden. Sie ermöglichen dynamische, aktuelle und präzise Antworten auf komplexe Anfragen.